# ch.paf.link Lite

Goal is to try a very lightweight ch.paf.link schema that would present a lower adoption barrier.

## Design Principles

- Activities group together what happened at one time (prov:atTime) or in one time interval (prov:startedAtTime and prov:endedAtTime) and was executed by agents described by a role that in itself clearly defines their contribution to the activity --> time and agend is mandatory
- Activities use entities as input and produce entities as output. A single entity can be produced and used at the same time by an activity. It is not per se defined, what input and output means but has to be defined in the application

In [82]:
from rdf_notebook import query_ttl

## Office Consultation

- Entities are not created, but only used
- Usage of entities is not repeated within activity stream
- No very specific chpaf classes, use paf classes with an additional general chpaf:OfficeConsultation class

In [83]:
ttl = """

@prefix chpaf: <https://ch.paf.link/> .
@prefix paf: <https://paf.link/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

# identifier entity - no creation
<https://politics.ld.admin.ch/parliamentary-affair/23.4022> a paf:IdentifierEntity, chpaf:ParliamentaryAffair ;
    schema:identifier "23.4022" .

# OCo identifier entity - no creation
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157> a paf:IdentifierEntity, chpaf:OfficeConsultation ;
    schema:identifier "OCo_2023.2157" .

# entity representing the consultation itself - no creation
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/entity> a paf:ConsultationEntity, chpaf:OfficeConsultation ;
    schema:description "Description of the consultation" ;
    schema:name "Title of the consultation" .

# activity for sending the consultation - with start and end time
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity> a paf:ConsultationActivity, chpaf:OfficeConsultation ;
    prov:startetAtTime "2023-12-18"^^xsd:date ;
    prov:endedAtTime "2024-01-15"^^xsd:date ;
    prov:used
        <https://politics.ld.admin.ch/parliamentary-affair/23.4022> ,
        <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157> ,
        <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/entity> ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/III.1.4> ;
        prov:hadRole chpaf:OfficeConsultationSender ;
    ] ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/II.1.2> ;
        prov:hadRole chpaf:OfficeConsultationReceiver ;
    ] ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/II.1.3> ;
        prov:hadRole chpaf:OfficeConsultationReceiver ;
    ] ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/II.1.4> ;
        prov:hadRole chpaf:OfficeConsultationReceiver ;
    ] .

# entity representing a comment - no creation
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-entity-1> a paf:CommentEntity, chpaf:OfficeConsultation ;
    schema:text "This is a comment..." .

# activity for commenting
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-activity-1> a paf:CommentActivity, chpaf:OfficeConsultation ;
    prov:wasInformedBy <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity> ;
    prov:atTime "2024-01-10T10:00:00"^^xsd:dateTime ;
    prov:used <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-entity-1> ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/II.1.2> ;
        prov:hadRole chpaf:OfficeConsultationCommentator ;
    ] .

# entity representing a comment - no creation
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-entity-2> a paf:CommentEntity, chpaf:OfficeConsultation ;
    schema:text "This is another comment..." .

# activity for commenting
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-activity-2> a paf:CommentActivity, chpaf:OfficeConsultation ;
    prov:wasInformedBy <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity> ;
    prov:atTime "2024-02-12T14:30:00"^^xsd:dateTime ;
    prov:used <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-entity-2> ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/II.1.3> ;
        prov:hadRole chpaf:OfficeConsultationCommentator ;
    ] .

"""

### Querying

- All comments to a certain consultation

In [84]:
query = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>

SELECT * WHERE {

    ?comment_entity a paf:CommentEntity, chpaf:OfficeConsultation ;
        ^prov:used/prov:wasInformedBy <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity> ;
        schema:text ?comment_text .
}

"""

query_ttl(ttl, query)

,comment_entity,comment_text
0,https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-entity-1,This is a comment...
1,https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-entity-2,This is another comment...


- Receivers that did comment

In [85]:
query = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>

SELECT ?agent WHERE {

    BIND(<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity> AS ?consultation_activity) .
    
    ?consultation_activity prov:qualifiedAssociation ?receiver_assoc .
    ?receiver_assoc prov:agent ?agent ;
        prov:hadRole chpaf:OfficeConsultationReceiver .

    ?comment_activity prov:wasInformedBy ?consultation_activity ; 
        prov:qualifiedAssociation ?commentator_assoc .
    ?commentator_assoc prov:agent ?agent ;
        prov:hadRole chpaf:OfficeConsultationCommentator .
}

"""

query_ttl(ttl, query)

,agent
0,https://ld.admin.ch/office/II.1.2
1,https://ld.admin.ch/office/II.1.3


- Receivers that did not comment

In [86]:
query = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>

SELECT ?agent WHERE {

    BIND(<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity> AS ?consultation_activity) .
    
    ?consultation_activity prov:qualifiedAssociation ?receiver_assoc .
    ?receiver_assoc prov:agent ?agent ;
        prov:hadRole chpaf:OfficeConsultationReceiver .
    
    MINUS {    
    
        ?comment_activity prov:wasInformedBy ?consultation_activity ; 
            prov:qualifiedAssociation ?commentator_assoc .
        ?commentator_assoc prov:agent ?agent ;
            prov:hadRole chpaf:OfficeConsultationCommentator .
    }

}

"""

query_ttl(ttl, query)

,agent
0,https://ld.admin.ch/office/II.1.4


- Check whether comment is within the correct date

In [87]:
query = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>

SELECT * WHERE {

    ?comment_activity a paf:CommentActivity, chpaf:OfficeConsultation ;
        prov:atTime ?comment_time ;
        prov:wasInformedBy ?consultation_activity .

    ?consultation_activity a paf:ConsultationActivity, chpaf:OfficeConsultation ;
        prov:startetAtTime ?start_date ;
        prov:endedAtTime ?end_date .

    # casting to xsd:dateTime for comparison works only after casting to string first in rdflib
    BIND(IF(?comment_time >= xsd:dateTime(STR(?start_date)) && ?comment_time <= xsd:dateTime(STR(?end_date)), "within timeframe", "out of timeframe") AS ?status)
}

"""

query_ttl(ttl, query)

,comment_time,comment_activity,start_date,status,end_date,consultation_activity
0,2024-01-10T10:00:00,https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-activity-1,2023-12-18,within timeframe,2024-01-15,https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity
1,2024-02-12T14:30:00,https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-activity-2,2023-12-18,out of timeframe,2024-01-15,https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity


## MoPo

In [88]:
ttl = """

@prefix chpaf: <https://ch.paf.link/> .
@prefix paf: <https://paf.link/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

# identifier entities - no creation
<https://politics.ld.admin.ch/parliamentary-affair/18.4276> a paf:IdentifierEntity, chpaf:ParliamentaryAffair ;
    schema:identifier "18.4276" .

<https://politics.ld.admin.ch/parliamentary-affair/18.4238> a paf:IdentifierEntity, chpaf:ParliamentaryAffair ;
    schema:identifier "18.4238" .

# mopo entity - no creation
<https://politics.ld.admin.ch/parliamentary-affair/18.4276/entity> a chpaf:ProceduralRequestEntity ;
    chpaf:proceduralRequestTitle "Erleichterter Informationsaustausch durch die Einführung von elektronischen Schnittstellen in der Bundesverwaltung"@de ;
    chpaf:proceduralRequestText "Der Bundesrat wird beauftragt, den direkten Informationsaustausch innerhalb der Bundesverwaltung..."@de ;
    chpaf:proceduralRequestType <https://ch.paf.link/vocabulary/procedural-request-type/motion> ;
    # new connex relation on entity level
    chpaf:proceduralRequestConnexTo <https://politics.ld.admin.ch/parliamentary-affair/18.4238> .

# mopo information entity - no creation
<https://politics.ld.admin.ch/procedural-request/information/2021/18.4238/entity> a chpaf:ProceduralRequestInformationEntity ;
    chpaf:proceduralRequestInformation "Die Motionen führten zu verschiedenen Umsetzungsarbeiten zur Schaffung und Veröffentlichung elektronischer Schnittstellen (API): ..."@de .

# information activity
<https://politics.ld.admin.ch/procedural-request/information/2021/18.4276/activity> a paf:InformationActivity, chpaf:ProceduralRequest, chpaf:ProceduralRequestConnex ;
    prov:qualifiedAssociation <https://politics.ld.admin.ch/procedural-request/information/2021/18.4276/activity/submitter> ;
    prov:atTime "2021"^^xsd:gYear ;
    prov:used 
    <https://politics.ld.admin.ch/parliamentary-affair/18.4276>, <https://politics.ld.admin.ch/parliamentary-affair/18.4276/entity>, 
    <https://politics.ld.admin.ch/procedural-request/information/2021/18.4238/entity> .

<https://politics.ld.admin.ch/procedural-request/information/2021/18.4276/activity/submitter> a prov:Association ;
    prov:agent "" ;
    prov:hadRole chpaf:ProceduralRequestInformationSubmitter .

"""

### Querying

- All Information from a year with connex

In [89]:
query = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>

SELECT ?curia ?title ?information ?connex WHERE {

    ?information_activity a paf:InformationActivity, chpaf:ProceduralRequest ;
        prov:atTime "2021"^^xsd:gYear ;
        prov:used/schema:identifier ?curia ;
        prov:used/chpaf:proceduralRequestTitle ?title ;
        prov:used/chpaf:proceduralRequestInformation ?information .

    OPTIONAL {
        ?information_activity prov:used/chpaf:proceduralRequestConnexTo/schema:identifier ?connex .
    }
}

"""

query_ttl(ttl, query)

,curia,title,information,connex
0,18.4276,Erleichterter Informationsaustausch durch die Einführung von elektronischen Schnittstellen in der Bundesverwaltung,Die Motionen führten zu verschiedenen Umsetzungsarbeiten zur Schaffung und Veröffentlichung elektronischer Schnittstellen (API): ...,18.4238


## Changes and Corrections

If traceable changes are required (one can always choose to just change an entity or an activity without leaving any trace), **entities and activities must not be changed after their creation / execution**.

One can always add information through additional entities that are used together with the original entity.

### Correcting Entities

- entities can be changed by a paf:EntityChangeActivity
- the old entity is used by the paf:EntityChangeActivity
- the new entity is generated by the paf:EntityChangeActivity
- if no new entity is created, this basically is a "revocation" of an entity
- a paf:EntityChangeActivity can only change one entity (because otherwise, there would be no clear connection between the old and the changed entity because there are multiple on both ends)

In [90]:
ttl = """

@prefix chpaf: <https://ch.paf.link/> .
@prefix paf: <https://paf.link/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix : <https://example.org/> .

# original entity - no creation
:entity1 a prov:Entity .

# some activity using the original entity
:activity1 a prov:Activity ;
    prov:used :entity1 .

# changing the entity - use of the original entity
:activity2 a prov:Activity, paf:EntityChangeActivity ;
    prov:used :entity1 ;
    prov:wasInformedBy :activity1 .

# new version of the entity created - creation by the change activity
:entity2 a prov:Entity ;
    prov:wasGeneratedBy :activity2 .

# another activity using the new entity
:activity3 a prov:Activity ;
    prov:used :entity2 ;
    prov:wasInformedBy :activity2 .

"""

### Querying

- changed entities and the change activity

In [91]:
query = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX : <https://example.org/>

SELECT ?old ?activity ?new WHERE {

    ?activity a paf:EntityChangeActivity ;
        prov:used ?old .

    ?new prov:wasGeneratedBy ?activity .
}

"""

query_ttl(ttl, query)

,old,activity,new
0,https://example.org/entity1,https://example.org/activity2,https://example.org/entity2


- activities that use a later changed entity (for checking if this activity needs further consideration after the entity was changed)

In [92]:
query = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX : <https://example.org/>

SELECT ?activity WHERE {

    ?ChangeActivity a paf:EntityChangeActivity ;
        prov:wasInformedBy+ ?activity . 
}

"""

query_ttl(ttl, query)

,activity
0,https://example.org/activity1


## Business Objects

If there is a need, to combine multiple entities and the fact, that some activities took place into a single "business object", this could be done also through querying: 

In [93]:
ttl = """

@prefix chpaf: <https://ch.paf.link/> .
@prefix paf: <https://paf.link/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

# identifier entity - no creation
<https://politics.ld.admin.ch/parliamentary-affair/23.4022> a paf:IdentifierEntity, chpaf:ParliamentaryAffair ;
    schema:identifier "23.4022" .

# OCo identifier entity - no creation
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157> a paf:IdentifierEntity, chpaf:OfficeConsultation ;
    schema:identifier "OCo_2023.2157" .

# entity representing the consultation itself - no creation
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/entity> a paf:ConsultationEntity, chpaf:OfficeConsultation ;
    schema:description "Description of the consultation" ;
    schema:name "Title of the consultation" .

# activity for sending the consultation - with start and end time
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity> a paf:ConsultationActivity, chpaf:OfficeConsultation ;
    prov:startetAtTime "2023-12-18"^^xsd:date ;
    prov:endedAtTime "2024-01-15"^^xsd:date ;
    prov:used
        <https://politics.ld.admin.ch/parliamentary-affair/23.4022> ,
        <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157> ,
        <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/entity> ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/III.1.4> ;
        prov:hadRole chpaf:OfficeConsultationSender ;
    ] ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/II.1.2> ;
        prov:hadRole chpaf:OfficeConsultationReceiver ;
    ] ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/II.1.3> ;
        prov:hadRole chpaf:OfficeConsultationReceiver ;
    ] ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/II.1.4> ;
        prov:hadRole chpaf:OfficeConsultationReceiver ;
    ] .

# entity representing a comment - no creation
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-entity-1> a paf:CommentEntity, chpaf:OfficeConsultation ;
    schema:text "This is a comment..." .

# activity for commenting
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-activity-1> a paf:CommentActivity, chpaf:OfficeConsultation ;
    prov:wasInformedBy <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity> ;
    prov:atTime "2024-01-10T10:00:00"^^xsd:dateTime ;
    prov:used <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-entity-1> ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/II.1.2> ;
        prov:hadRole chpaf:OfficeConsultationCommentator ;
    ] .

# entity representing a comment - no creation
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-entity-2> a paf:CommentEntity, chpaf:OfficeConsultation ;
    schema:text "This is another comment..." .

# activity for commenting
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-activity-2> a paf:CommentActivity, chpaf:OfficeConsultation ;
    prov:wasInformedBy <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity> ;
    prov:atTime "2024-02-12T14:30:00"^^xsd:dateTime ;
    prov:used <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-entity-2> ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/II.1.3> ;
        prov:hadRole chpaf:OfficeConsultationCommentator ;
    ] .

"""

- status of the consultation depending on activity and date

In [94]:
query = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX : <https://example.org/>

SELECT ?curia_id ?oco_id ?start ?stop ?status ?name ?description WHERE {

    BIND ("2024-01-01"^^xsd:date AS ?now) .
    
    ?activity a paf:ConsultationActivity, chpaf:OfficeConsultation ;
        prov:startetAtTime ?start ;
        prov:endedAtTime ?stop ;
        prov:used ?curia_id_entity, ?oco_id_entity, ?oco_entity .
    
    ?oco_id_entity a paf:IdentifierEntity, chpaf:OfficeConsultation ;
        schema:identifier ?oco_id .

    ?curia_id_entity a paf:IdentifierEntity, chpaf:ParliamentaryAffair ;
        schema:identifier ?curia_id .

    ?oco_entity a paf:ConsultationEntity, chpaf:OfficeConsultation ;
        schema:name ?name ;
        schema:description ?description .

    BIND(
        IF(?now < ?start, "registered",
            IF(?now > ?stop, "finished", "taking place")
            ) AS ?status
    )

}

"""

query_ttl(ttl, query)

,curia_id,oco_id,start,stop,status,name,description
0,23.4022,OCo_2023.2157,2023-12-18,2024-01-15,taking place,Title of the consultation,Description of the consultation


- consultation answers and their timeliness, tabular result transformed to JSON

In [95]:
query = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX : <https://example.org/>

SELECT ?oco_id ?commentator ?comment ?comment_status WHERE {
    
    ?activity a paf:ConsultationActivity, chpaf:OfficeConsultation ;
        prov:startetAtTime ?start ;
        prov:endedAtTime ?stop ;
        prov:used ?oco_id_entity .
    
    ?oco_id_entity a paf:IdentifierEntity, chpaf:OfficeConsultation ;
        schema:identifier ?oco_id .

    ?answer a paf:CommentActivity, chpaf:OfficeConsultation ;
        prov:wasInformedBy ?activity ;
        prov:atTime ?comment_time ;
        prov:qualifiedAssociation/prov:agent ?commentator ;
        prov:used/schema:text ?comment .

    BIND(IF(?comment_time <= xsd:dateTime(STR(?stop)), "in time", "late") AS ?comment_status)
    
}

"""

df = query_ttl(ttl, query)

display(df)

result = {
    key: group.drop(columns="oco_id").to_dict(orient="records")
    for key, group in df.groupby("oco_id")
}

import json
print(json.dumps(result, indent=2, ensure_ascii=False))

,oco_id,commentator,comment,comment_status
0,OCo_2023.2157,https://ld.admin.ch/office/II.1.2,This is a comment...,in time
1,OCo_2023.2157,https://ld.admin.ch/office/II.1.3,This is another comment...,late


{
  "OCo_2023.2157": [
    {
      "commentator": "https://ld.admin.ch/office/II.1.2",
      "comment": "This is a comment...",
      "comment_status": "in time"
    },
    {
      "commentator": "https://ld.admin.ch/office/II.1.3",
      "comment": "This is another comment...",
      "comment_status": "late"
    }
  ]
}
